In [1]:
import pandas as pd
import tensorflow as tf
import nltk
import re
from keras.models import Sequential
from keras import layers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from tensorflow.python.client import device_lib
from nltk import word_tokenize          
from nltk.stem import porter, lancaster, snowball


Using TensorFlow backend.


In [2]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15419892429949475504
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9204972913
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3270026993092832017
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
with tf.device("gpu:0"):
   print("tf.keras code in this scope will run on GPU")

tf.keras code in this scope will run on GPU


In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train

,label,text
0,1,"Henry Thomas showed a restraint, even when the..."
1,1,"This movie starts out brisk, has some slow mom..."
2,1,Castle of Blood is a good example of the quali...
3,1,I viewed the movie together with a homophobic ...
4,1,"The ""Men in White"" movie is definitely one of ..."
...,...,...
9995,0,Although Bette Davis did a WONDERFUL job as Mi...
9996,0,I am working my way through the Chilling Class...
9997,0,Gods...where to start. I was only able to stom...
9998,0,It's hard to say what was the worst thing abou...


In [6]:
cont_vect = TfidfVectorizer( ngram_range = (1,4))
X_train = cont_vect.fit_transform(train.text)
X_train

<10000x4482509 sparse matrix of type '<class 'numpy.float64'>'
	with 7929212 stored elements in Compressed Sparse Row format>

In [7]:
X_test = cont_vect.transform(test.text)
y_train = train.label

# Keras

In [8]:
input_dim = X_train.shape[1]  # Number of features

model2 = Sequential()
model2.add(layers.Dense(50, input_dim=input_dim, activation='relu'))
model2.add(layers.Dense(50, activation='softmax'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                224125500 
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 224,128,101
Trainable params: 224,128,101
Non-trainable params: 0
_________________________________________________________________


In [9]:
myfit = model2.fit(X_train, y_train,epochs=1,verbose=1, batch_size=10)

Epoch 1/1
10000/10000 [==============================] - 449s 45ms/step - loss: 0.5849 - accuracy: 0.7767


In [ ]:
prediction = model2.predict_classes(X_test)

In [ ]:
prediction

In [ ]:
flatt = prediction.flatten()
flatt

In [ ]:
pd.DataFrame({"Id": test.Id, "Category": flatt}).to_csv("prediction_ANN.csv", index=False)